# Memory steering · Plotting harness walkthrough

`src/memory_steering/plot_memory_steering.py` converts experiment CSVs into figures and verdict logs. This notebook fabricates a tiny CSV, parses it, and shows how to invoke the plotting helper.


In [ ]:
import sys
from pathlib import Path


def _ensure_repo_on_path():
    cwd = Path.cwd().resolve()
    candidates = [cwd, cwd.parent, cwd.parent.parent]
    for candidate in candidates:
        utils_py = candidate / 'notebooks' / 'utils.py'
        if utils_py.exists():
            sys.path.insert(0, str(candidate))
            return candidate
    return cwd

_ensure_repo_on_path()

from notebooks.utils import ensure_repo_path, preview_json, allocate_artifacts

repo_root = ensure_repo_path()
print(f"Using repo root: {repo_root}")



In [ ]:
from pathlib import Path
from textwrap import dedent

sample_csv = dedent('''
# Junction logistic (CSV): Theta*Delta_m, P(A)
-1.0,0.18
0.0,0.50
1.0,0.82

# Curvature scaling (CSV): Theta*|grad m|, mean(kappa_path)
0.05,0.048
0.10,0.097

# Curvature scaling signed (CSV): X, mean_kappa, se_kappa, seed, sign_id
0.10,0.096,0.004,77,0
0.10,0.094,0.005,77,1

# Stability band (CSV|dose_model=scale_R): D_a, Lambda, Gamma, Retention, Fidelity_w, Fidelity_end, Fidelity_shuffle_end, Fidelity_edge_end, AUC_end, SNR_end, AUPRC_topk, BPER
1.0,0.5,0.2,0.74,0.88,0.81,0.02,0.78,0.71,5.2,0.63,0.12
''')
results_path = repo_root / 'notebooks' / '_demo_memory_results.csv'
results_path.write_text(sample_csv)
print(f"Wrote demo CSV to {results_path}")


In [ ]:
try:
    from src.memory_steering.plot_memory_steering import parse_results, plot_all
except ModuleNotFoundError:
    from notebooks.utils import ensure_repo_path as _vdm_ensure_repo_path
    _vdm_ensure_repo_path()
    import importlib
    importlib.invalidate_caches()
    from src.memory_steering.plot_memory_steering import parse_results, plot_all


Jx, Jp, Cx, Cy, SB, Sx, Smy, Sse, Sseed, Ssign = parse_results(str(results_path))
print(preview_json({
    "junction_points": len(Jx),
    "curvature_points": len(Cx),
    "stability_shape": list(SB.shape) if SB.size else None,
}))

artifacts = allocate_artifacts("memory_steering", "plotting_notebook_demo")
plot_all(str(results_path), outdir=str(artifacts["figure"].parent))
print(f"Summary figure directory: {artifacts['figure'].parent}")


In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(11, 4))
axes[0].plot(Jx, Jp, 'o-')
axes[0].set_xlabel('Theta * Delta m')
axes[0].set_ylabel('P(A)')
axes[0].set_title('Junction logistic (demo)')
axes[1].plot(Cx, Cy, 'o', color='tab:green')
axes[1].set_xlabel('Theta * |grad m|')
axes[1].set_ylabel('mean(kappa_path)')
axes[1].set_title('Curvature scaling (demo)')
for ax in axes:
    ax.grid(True, alpha=0.3)
fig.tight_layout()
plt.show()


## Notebook checklist

* The demo CSV mimics the section tags the plotting harness expects; swap in a real sweep output when available.
* `parse_results` returns NumPy arrays ready for bespoke analysis if you want to bypass the canned plots.
* `plot_all` writes figures plus a verdict JSON using the shared `io_paths` routing.
* Delete `_demo_memory_results.csv` once you no longer need the synthetic sample.
